<a href="https://colab.research.google.com/github/bhargavpirates/Machine-Language-Translation-/blob/master/HIndi_English_Machine_Translation_Using_Encoder_Decoder_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HIndi-English Machine Translation Using Encoder  Decoder LSTM Model.ipynb

1.  Here we have Hindi Sentence documents, we need to convert hindi sentence documents to english sentence documents .so, we need to design  Sequenc2Sequence model
2.  Sequenc2Sequence models in deeplearning is RNN,LSTM,GRU

### Most common Aarchitecture used to build Seq2Seq models is the Encoder Decoder architecture


### mounting data from google Drive to Google Collab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/parallel/hindi.txt" "hindi.txt"
!cp "/content/drive/My Drive/parallel/english.txt" "english.txt"

In [0]:
!nvidia-smi

Fri Sep  6 07:20:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### importing libraries

In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

### Reading Text Data which is in .txt file and stoing them in a varible

In [0]:
with open("hindi.txt",encoding='utf-8') as f:
    hindi_txt=f.read().split('\n')
with open("english.txt",encoding='utf-8') as f:
    english_txt=f.read().split('\n')

### Constructing DataFrame ontop of txt varibles

In [0]:
df= pd.DataFrame({"hin":hindi_txt,"eng":english_txt})
print(df.shape)

(1561841, 2)


In [0]:

df=df[:100000]

## Preprocessing Data 

In [0]:
#covert all words in a documents to lower case
df.hin=df.hin.apply(lambda x: x.lower())
df.eng=df.eng.apply(lambda x: x.lower())
df.hin=df.hin.apply(lambda x: re.sub("'", '', x))
df.eng=df.eng.apply(lambda x: re.sub("'", '', x))

# Set of all special characters
set_punc = set(string.punctuation
# Remove all the special characters
df.hin=df.hin.apply(lambda x: ''.join(ch for ch in x if ch not in set_punc))
df.eng=df.eng.apply(lambda x: ''.join(ch for ch in x if ch not in set_punc))

In [0]:
df.shape

(100000, 2)

In [0]:
# length of sentence must be greater then five
indexNames = df[ (df['hin'].str.len()  <= 5) | (df['eng'].str.len()  <= 5) ].index
df.drop(indexNames , inplace=True)

In [0]:
df.shape

(88884, 2)

In [0]:
# Removing Puncations in hindi txt
from string import punctuation
p = re.compile(r'[\s{}]+'.format(re.escape(punctuation)))
df.hin=df.hin.apply(lambda x: re.sub(p,' ', x))

In [0]:
# Remove all numbers from English text
remove_digits = str.maketrans('', '', digits)
df.eng=df.eng.apply(lambda x: x.translate(remove_digits))
df.eng=df.eng.apply(lambda x: re.sub("[0-9]", "", x))

# Remove all numbers from Hindi text
df.hin=df.hin.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
df.hin=df.hin.apply(lambda x: re.sub("[0123456789]", "", x))

# Remove all English words from  Hindi text
df.hin=df.hin.apply(lambda x: re.sub("[A-Za-z]", "", x))

df.hin=df.hin.apply(lambda x: x.strip())
df.eng=df.eng.apply(lambda x: x.strip())
df.hin=df.hin.apply(lambda x: re.sub(" +", " ", x))
df.eng=df.eng.apply(lambda x: re.sub(" +", " ", x))

### Add [go] and [stop] tokens to target sequences
   * [go] at the starting of sentence English  Sentence
   * [Start] at the Ending of target English Sentence

In [0]:
# Add [go] and [stop] tokens to target sequences
df.eng=df.eng.apply(lambda x : '[go] ' + x + ' [stop]')

In [0]:
# Remove extra spaces# Vocabulary of English
all_eng_words=set()
for eng in df.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Hindi 
all_hin_words=set()
for hin in df.hin:
    for word in hin.split():
        if word not in all_hin_words:
            all_hin_words.add(word)




#### Finding maximum sentence length in English text Corpus and HIndi Text Corpus

In [0]:
# Max Length of source[Hindi] sequence
lenght_list=[]
for l in df.hin:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print("Maximum length of sentence in whole hindi Documents ::: {}".format(max_length_src))

# Max Length of target[English] sequence
lenght_list=[]
for l in df.eng:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print("Maximum length of sentence in whole English Documents ::: {}".format(max_length_tar))

Maximum length of sentence in whole hindi Documents ::: 111
Maximum length of sentence in whole English Documents ::: 136


### Storing  unique words[tokens] in a varibles for  hindi corpus  and English corpus  

In [0]:
input_words = sorted(list(all_hin_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_hin_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(6290, 5615)

In [0]:
num_decoder_tokens += 1
num_encoder_tokens += 1

### convert unique tokens in form of (tokens,index) tuple


In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

### convert unique tokens in form of (tokens,index) tuple

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [0]:
df = shuffle(df)
df.head(10)

,hin,eng
34564,इस सीडी या डीवीडी डिस्क की कॉपी करें,[go] create a copy of this cd or dvd [stop]
63746,वेब विकास,[go] web development [stop]
16422,पूर्ण के साथ स्थिति कोड नहीं,[go] completed unsuccessfully with status code...
27585,औज़ार युक्ति,[go] tool tip [stop]
92857,क्या आप निश्चित हैं कि आप प्रारूप में संदेश भे...,[go] are you sure you want to send a message i...
61781,पसंद निर्यात करें,[go] export bookmarks [stop]
16740,बिल्ड माड्यूल,[go] build module [stop]
51279,वर्तनीजांच भाषा,[go] enable spell checking for languages [stop]
43191,फ़ॉन्ट उपपिक्सल अनुक्रम,[go] font subpixel order [stop]
5059,को एक ईंट का इक्का पर ले जाएँ,[go] move a onto the ace of diamonds [stop]


## Explaning Encoder and Decoder Architecture
* for encoder we can use any deeplearning models like (MLP,CNN,RNN,LSTM,GRU)
* for decoder we also can use any deeplearning models like (MLP,CNN,RNN,LSTM,GRU)
*  BUt for this word2word sequence problem for both  encoder and the decoder are typically use LSTM models (or sometimes GRU models)
* Encoder Model :: 
  *  Encoder reads the input sequence and summarizes the information in something called as the internal state vectors (in case of LSTM these are called as the hidden state and cell state vectors). We discard the outputs of the encoder and only preserve the internal states.
* Decoder MOdel ::
  * Decoder is an LSTM whose initial states are initialized to the final states of the Encoder LSTM. Using these initial states, decoder starts generating the output sequence.
  * The decoder behaves a bit differently during the training and inference procedure. During the training, we use a technique call teacher forcing which helps to train the decoder faster. During inference, the input to the decoder at each time step is the output from the previous time step.
  
* Final intutation :: 
  * Intuitively, the encoder summarizes the input sequence into state vectors (sometimes also called as Thought vectors), which are then fed to the decoder which starts generating the output sequence given the Thought vectors
  * The decoder is just a language model conditioned on the initial states.
  

### Specifing Traning and Testing Data
* here for machine translation we have taken 
  * INPUT DATA as HIndi Sentence
  * OUTPUT DATA as English Sentence

### Spliting  Data into Train and Text in ratio of 0.33

In [0]:
# Train - Test Split
X, y = df.hin, df.eng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(59552,) (29332,)
(59552,) (29332,)


In [0]:
y_train[:3]

16866                                 [go] cvsroot [stop]
52442    [go] play a sound when a contact logs out [stop]
24738               [go] cvs remove file directory [stop]
Name: eng, dtype: object

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    #print(word)
                    #print(target_text.split())
                    #print(t)
                    #print(len(target_text.split()))
                    if t<len(target_text.split())-1:
                        #print("going")
                        decoder_input_data[i, t] = str(target_token_index[word]) # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the "[go]" token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Encoder - Decoder Model Architecture

#### Specifying Embedding output dimension

In [0]:
latent_dim = 50

### Encoder LSTM Model 
 1. need to convert encoder input data into vectorization form .for that we are using Embedding Function
 2. Embedding Function 
    * coverts each value into vector form
    * for that we need to train  Embedding Model by specifying  input_dimension  , output_dimension with input_data
    * input_dimension  = num_encoder_tokens
    * output_dimension = latent_dim
    * input_data       = Encoder_input_data
 3. Specifying LSTM model 
    *  with units = 50
    * return_state = True
 4. Storing final states of Encoder_LSTM model in a varible  encoder_states

In [0]:
#Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### Decoder LSTM Model 
 1. need to convert Decoder input data into vectorization form .for that we are using Embedding Function
 2. Embedding Function 
    * coverts each value into vector form
    * for that we need to train  Embedding Model by specifying  input_dimension  , output_dimension with input_data
    * input_dimension  = num_decoder_tokens
    * output_dimension = latent_dim
    * input_data       = Encoder_input_data
 3. Specifying LSTM model 
    *  with units = 50
    * return_state = True
    * return_sequences=True [ because we want to get full output sequence]
 4.  For DecoderLTSM input Data is Final_States_of_Encoder and decoder_inputData
 5.  Applying Softmax Function on top of DecoderOutput 
      

In [0]:
# Set up the decoder, using 'ncoder_states as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### model Compiling 
1. using lossFunction as 'categorical_crossentropy'
1. we can use any of optimizaion algorithms like 'rmsprop','adm','Adadelta' etc
2. metric have choose for this accuracy


In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

#### Specifying len_training_Samples,len_validation_Samples,Batch_Size and Epochs 

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 20

### Running Model

In [0]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                            steps_per_epoch = train_samples//batch_size,
                            epochs=epochs,
                            validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                            validation_steps = val_samples//batch_size)

Epoch 1/20
465/465 [==============================] - 552s 1s/step - loss: 5.9172 - acc: 0.1816 - val_loss: 5.4882 - val_acc: 0.2059
Epoch 2/20
465/465 [==============================] - 541s 1s/step - loss: 5.2612 - acc: 0.2172 - val_loss: 5.0921 - val_acc: 0.2300
Epoch 3/20
465/465 [==============================] - 541s 1s/step - loss: 4.9225 - acc: 0.2432 - val_loss: 4.8106 - val_acc: 0.2571
Epoch 4/20
465/465 [==============================] - 539s 1s/step - loss: 4.6263 - acc: 0.2738 - val_loss: 4.5171 - val_acc: 0.2904
Epoch 5/20
465/465 [==============================] - 540s 1s/step - loss: 4.3305 - acc: 0.3087 - val_loss: 4.2571 - val_acc: 0.3224
Epoch 6/20
465/465 [==============================] - 540s 1s/step - loss: 4.0662 - acc: 0.3408 - val_loss: 4.0197 - val_acc: 0.3523
Epoch 7/20
465/465 [==============================] - 538s 1s/step - loss: 3.8284 - acc: 0.3723 - val_loss: 3.8067 - val_acc: 0.3803
Epoch 8/20
465/465 [==============================] - 539s 1s/step - 

### Saving weights of model in nwt_weights.h5

In [0]:
model.save('nmt_weights.h5')

In [0]:
!ls

drive  english.txt  hindi.txt  nmt_weights.h5  sample_data


### Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['[go]']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '[stop]' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Model Testing

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [0]:
k+=1
print(k)
(input_seq, actual_output), _ = next(train_gen)
#print(input_seq)
#print(actual_output)
decoded_sentence = decode_sequence(input_seq)
print(decoded_sentence)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][4:-6])

print('Predicted English Translation:', decoded_sentence[:-6])

16
 complete log [stop]
Input Hindi sentence: प्रतिशत समाप्तः र्ण
Actual English Translation:  percent complete i 
Predicted English Translation:  complete log 
